# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import gzip
import json
import datetime

In [2]:
pd.set_option('display.float_format', '{:.0f}'.format)

# Cleaning brands.json.gz

In [3]:
br_data = [json.loads(line) for line in gzip.open("brands.json.gz", "r")]
br_df = pd.json_normalize(br_data)
# Renaming column names
br_df_cleaned = br_df.rename(columns={'_id.$oid': 'brand_id', 'cpg.$id.$oid': 'cpg_id', 'cpg.$ref': 'cpg_reference'})

In [4]:
br_df_cleaned.head()

,barcode,category,categoryCode,name,topBrand,brand_id,cpg_id,cpg_reference,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


In [5]:
print(br_df_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   barcode        1167 non-null   object
 1   category       1012 non-null   object
 2   categoryCode   517 non-null    object
 3   name           1167 non-null   object
 4   topBrand       555 non-null    object
 5   brand_id       1167 non-null   object
 6   cpg_id         1167 non-null   object
 7   cpg_reference  1167 non-null   object
 8   brandCode      933 non-null    object
dtypes: object(9)
memory usage: 82.2+ KB
None


### Saving the cleaned Brand dataframe as a Newline Delimited JSON file to import into BigQuery Database

In [6]:
br_df_cleaned.to_json('brand_cleaned.jsonl', orient='records', lines=True)  # Save as records format

In [7]:
print(br_df_cleaned.columns)

Index(['barcode', 'category', 'categoryCode', 'name', 'topBrand', 'brand_id',
       'cpg_id', 'cpg_reference', 'brandCode'],
      dtype='object')


In [8]:
row_count = br_df_cleaned.shape[0]
print(f"Number of rows in the DataFrame: {row_count}")

Number of rows in the DataFrame: 1167


# Cleaning receipts.json.gz

In [9]:
rec_data = [json.loads(line) for line in gzip.open("receipts.json.gz", "r")]
rec_df = pd.DataFrame(rec_data)
rec_df.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5,{'$date': 1609632000000},1,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052


### Checking if any columns have null values

In [10]:
print(rec_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             609 non-null    object 
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               684 non-null    object 
 14  userId                  

### Inspecting receipts with only date and IDs to find an example

In [11]:
null_receipts = rec_df[rec_df.isnull().any(axis=1)]
print(null_receipts)

                                       _id  bonusPointsEarned  \
2     {'$oid': '5ff1e1f10a720f052300057a'}                  5   
12    {'$oid': '5ff1e1b60a7214ada100055c'}                150   
13    {'$oid': '5f9c74f70a7214ad07000037'}                750   
15    {'$oid': '5ff1e1e90a7214ada1000569'}                NaN   
17    {'$oid': '5ff1e1b40a7214ada100055b'}                750   
...                                    ...                ...   
1114  {'$oid': '603cc0630a720fde100003e6'}                 25   
1115  {'$oid': '603d0b710a720fde1000042a'}                NaN   
1116  {'$oid': '603cf5290a720fde10000413'}                NaN   
1117  {'$oid': '603ce7100a7217c72c000405'}                 25   
1118  {'$oid': '603c4fea0a7217c72c000389'}                NaN   

                                bonusPointsEarnedReason  \
2                            All-receipts receipt bonus   
12    Receipt number 5 completed, bonus point schedu...   
13    Receipt number 1 completed, bonus po

### Example for a receipt with only date and IDs

In [12]:
if rec_df.iloc[1118].isnull().any():
    print(rec_df.iloc[1118])
else:
    print("Row 1118 does not contain NULL values.")

_id                        {'$oid': '603c4fea0a7217c72c000389'}
bonusPointsEarned                                           NaN
bonusPointsEarnedReason                                     NaN
createDate                             {'$date': 1614565354962}
dateScanned                            {'$date': 1614565354962}
finishedDate                                                NaN
modifyDate                             {'$date': 1614565354962}
pointsAwardedDate                                           NaN
pointsEarned                                                NaN
purchaseDate                                                NaN
purchasedItemCount                                          NaN
rewardsReceiptItemList                                      NaN
rewardsReceiptStatus                                  SUBMITTED
totalSpent                                                  NaN
userId                                 5fc961c3b8cfca11a077dd33
Name: 1118, dtype: object


### Function to Extract value if it's a dictionary with '$date' as the key

In [13]:
def extract_date(x):
    if isinstance(x, dict) and '$date' in x:
        return x['$date']
    return None 

### Aplying function to all the date columns in receipts dataframe

In [14]:
rec_df_cleaned = rec_df.dropna(subset=['rewardsReceiptItemList']).copy()
rec_id_list = [i['$oid'] for i in rec_df_cleaned._id]
rec_df_cleaned._id = rec_id_list
rec_df_cleaned['createDate'] = rec_df_cleaned['createDate'].apply(extract_date)
rec_df_cleaned['dateScanned'] = rec_df_cleaned['dateScanned'].apply(extract_date)
rec_df_cleaned['finishedDate'] = rec_df_cleaned['finishedDate'].apply(extract_date)
rec_df_cleaned['modifyDate'] = rec_df_cleaned['modifyDate'].apply(extract_date)
rec_df_cleaned['pointsAwardedDate'] = rec_df_cleaned['pointsAwardedDate'].apply(extract_date)
rec_df_cleaned['purchaseDate'] = rec_df_cleaned['purchaseDate'].apply(extract_date)
rec_df_cleaned.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,1609687531000,1609687536000,1609687531000,500.0,1609632000000,5,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1609687483000,1609687488000,1609687483000,150.0,1609601083000,2,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5,All-receipts receipt bonus,1609687537000,1609687537000,NaN,1609687542000,NaN,5,1609632000000,1,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5,All-receipts receipt bonus,1609687534000,1609687534000,1609687534000,1609687539000,1609687534000,5.0,1609632000000,4,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5,All-receipts receipt bonus,1609687506000,1609687506000,1609687511000,1609687511000,1609687506000,5.0,1609601106000,2,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052


### Dropping "rewardsReceiptItemList" which is a dictionary array and to be added in Junction table

In [15]:
rec_df = rec_df.drop(columns=['rewardsReceiptItemList'])

In [16]:
row_count = rec_df_cleaned.shape[0]
print(f"Number of rows in the DataFrame: {row_count}")

Number of rows in the DataFrame: 679


### Saving the cleaned Receipt dataframe as a Newline Delimited JSON file to import into BigQuery Database

In [17]:
rec_df_cleaned.to_json('receipt_cleaned.jsonl', orient='records', lines=True)  # Save as records format

### Function to convert date columns in unix timestamp format to datetime format

In [18]:
def convert_unix_to_datetime(unix_timestamp):
    try:
        if pd.isna(unix_timestamp):  # Check for NaN
            return None 
        if unix_timestamp is None:
            return None  # Return None for missing timestamps
        return datetime.datetime.fromtimestamp(unix_timestamp / 1000)  # Convert milliseconds to seconds
    except Exception as e:
        print(f"Error converting timestamp {unix_timestamp}: {e}")
        return None 

### Aplying function to all the date columns in Receipt dataframe

In [19]:
rec_df_cleaned['createDate'] = rec_df_cleaned['createDate'].apply(convert_unix_to_datetime)
rec_df_cleaned['dateScanned'] = rec_df_cleaned['dateScanned'].apply(convert_unix_to_datetime)
rec_df_cleaned['finishedDate'] = rec_df_cleaned['finishedDate'].apply(convert_unix_to_datetime)
rec_df_cleaned['modifyDate'] = rec_df_cleaned['modifyDate'].apply(convert_unix_to_datetime)
rec_df_cleaned['pointsAwardedDate'] = rec_df_cleaned['pointsAwardedDate'].apply(convert_unix_to_datetime)
rec_df_cleaned['purchaseDate'] = rec_df_cleaned['purchaseDate'].apply(convert_unix_to_datetime)

In [20]:
rec_df_cleaned.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500,"Receipt number 2 completed, bonus point schedu...",2021-01-03 09:25:31,2021-01-03 09:25:31,2021-01-03 09:25:31,2021-01-03 09:25:36,2021-01-03 09:25:31,500.0,2021-01-02 18:00:00,5,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150,"Receipt number 5 completed, bonus point schedu...",2021-01-03 09:24:43,2021-01-03 09:24:43,2021-01-03 09:24:43,2021-01-03 09:24:48,2021-01-03 09:24:43,150.0,2021-01-02 09:24:43,2,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5,All-receipts receipt bonus,2021-01-03 09:25:37,2021-01-03 09:25:37,NaT,2021-01-03 09:25:42,NaT,5,2021-01-02 18:00:00,1,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5,All-receipts receipt bonus,2021-01-03 09:25:34,2021-01-03 09:25:34,2021-01-03 09:25:34,2021-01-03 09:25:39,2021-01-03 09:25:34,5.0,2021-01-02 18:00:00,4,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5,All-receipts receipt bonus,2021-01-03 09:25:06,2021-01-03 09:25:06,2021-01-03 09:25:11,2021-01-03 09:25:11,2021-01-03 09:25:06,5.0,2021-01-02 09:25:06,2,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052


In [21]:
print(rec_df_cleaned.columns)

Index(['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate',
       'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate',
       'pointsEarned', 'purchaseDate', 'purchasedItemCount',
       'rewardsReceiptItemList', 'rewardsReceiptStatus', 'totalSpent',
       'userId'],
      dtype='object')


# Cleaning receipts.json.gz to create dataset for Junction table (Itemized Receipts)

In [22]:
itemized_receipts_df = pd.DataFrame(rec_df_cleaned)

### rewardsReceiptItemList column has all the columns needed for the junction table in a dictionary array, flattening the column to create a column for each key

### Barcode column will be used to join brand and Itemized Receipt Table

In [23]:
itemized_receipts_flattened_df = itemized_receipts_df.explode('rewardsReceiptItemList')
print(itemized_receipts_flattened_df.columns)

Index(['_id', 'bonusPointsEarned', 'bonusPointsEarnedReason', 'createDate',
       'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate',
       'pointsEarned', 'purchaseDate', 'purchasedItemCount',
       'rewardsReceiptItemList', 'rewardsReceiptStatus', 'totalSpent',
       'userId'],
      dtype='object')


In [24]:
itemized_receipts_flattened_items_df = pd.json_normalize(itemized_receipts_flattened_df['rewardsReceiptItemList'])
print(itemized_receipts_flattened_items_df.columns)

Index(['barcode', 'description', 'finalPrice', 'itemPrice', 'needsFetchReview',
       'partnerItemId', 'preventTargetGapPoints', 'quantityPurchased',
       'userFlaggedBarcode', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity', 'needsFetchReviewReason',
       'pointsNotAwardedReason', 'pointsPayerId', 'rewardsGroup',
       'rewardsProductPartnerId', 'userFlaggedDescription',
       'originalMetaBriteBarcode', 'originalMetaBriteDescription', 'brandCode',
       'competitorRewardsGroup', 'discountedItemPrice',
       'originalReceiptItemText', 'itemNumber',
       'originalMetaBriteQuantityPurchased', 'pointsEarned', 'targetPrice',
       'competitiveProduct', 'originalFinalPrice',
       'originalMetaBriteItemPrice', 'deleted', 'priceAfterCoupon',
       'metabriteCampaignId'],
      dtype='object')


### Resetting indexes to concatenate and create one dataframe

### Using '_id' column from Receipt dataframe so that it can be used to join Receipt table and Itemized Receipt table and act as Primary Key for Itemized Receipt table

In [25]:
itemized_receipts_flattened_df.reset_index(drop=True, inplace=True)
itemized_receipts_flattened_df.reset_index(drop=True, inplace=True)

itemized_receipts_combined_df = pd.concat(
    [itemized_receipts_flattened_df[['_id']], itemized_receipts_flattened_items_df], 
    axis=1,
    ignore_index=False
)
itemized_receipts_combined_df.head()

,_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1,028400642255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Saving the cleaned Itemized Receipts dataframe as a Newline Delimited JSON file to import into BigQuery Database

In [26]:
itemized_receipts_combined_df.to_json('itemized_receipt_cleaned.jsonl', orient='records', lines=True)  # Save as records format

In [27]:
row_count = itemized_receipts_combined_df.shape[0]
print(f"Number of rows in the DataFrame: {row_count}")

Number of rows in the DataFrame: 6941


# Cleaning users.json (Unzipped File)

In [28]:
user_data = []

with open("C:\\Users\\gadas\\Downloads\\users.json\\users.json", "r", encoding="utf-8") as f:
    total_lines = 0
    valid_lines = 0

    for line_number, line in enumerate(f, start=1):
        total_lines += 1
        line = line.strip()  # Removing any leading/trailing whitespace
        
        if not line:
            print(f"Skipping line {line_number}: Empty line")
            continue
        
        try:
            raw_data = json.loads(line)
            valid_lines += 1  # Count valid lines

            cleaned_data = {
                'id': raw_data['_id']['$oid'],  # Extract oid
                'active': raw_data.get('active', False),  # Used .get() to avoid KeyError
                'createdDate': raw_data['createdDate']['$date'],  
                'lastLogin': raw_data.get('lastLogin', {}).get('$date', 0) if 'lastLogin' in raw_data else None, 
                'role': raw_data.get('role', None),  # Use .get() for role
                'signUpSource': raw_data.get('signUpSource', None),  
                'state': raw_data.get('state', None) 
            }
            user_data.append(cleaned_data) 
            
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON on line {line_number}: {line}")
            continue  
        except Exception as e:
            print(f"Unexpected error on line {line_number}: {line} - {e}")
            continue  

user_df_cleaned = pd.DataFrame(user_data)

# Display statistics
print(f"Total lines read: {total_lines}")
print(f"Valid lines processed: {valid_lines}")
print(f"Total records in DataFrame: {len(user_df_cleaned)}")

print(user_df_cleaned.head())

Total lines read: 495
Valid lines processed: 495
Total records in DataFrame: 495
                         id  active    createdDate     lastLogin      role  \
0  5ff1e194b6a9d73a3a9f1052    True  1609687444800 1609687537858  consumer   
1  5ff1e194b6a9d73a3a9f1052    True  1609687444800 1609687537858  consumer   
2  5ff1e194b6a9d73a3a9f1052    True  1609687444800 1609687537858  consumer   
3  5ff1e1eacfcf6c399c274ae6    True  1609687530554 1609687530597  consumer   
4  5ff1e194b6a9d73a3a9f1052    True  1609687444800 1609687537858  consumer   

  signUpSource state  
0        Email    WI  
1        Email    WI  
2        Email    WI  
3        Email    WI  
4        Email    WI  


In [29]:
user_df_cleaned.head()

,id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1609687537858,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1609687537858,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1609687537858,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,1609687530554,1609687530597,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1609687537858,consumer,Email,WI


In [30]:
row_count = user_df_cleaned.shape[0]
print(f"Number of rows in the DataFrame: {row_count}")

Number of rows in the DataFrame: 495


### Aplying convert_unix_to_datetime function to all the date columns in user dataframe

In [31]:
user_df_cleaned['createdDate'] = user_df_cleaned['createdDate'].apply(convert_unix_to_datetime)
user_df_cleaned['lastLogin'] = user_df_cleaned['lastLogin'].apply(convert_unix_to_datetime)

In [32]:
user_df_cleaned.head()

,id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 09:24:04.800,2021-01-03 09:25:37.858,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 09:24:04.800,2021-01-03 09:25:37.858,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 09:24:04.800,2021-01-03 09:25:37.858,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 09:25:30.554,2021-01-03 09:25:30.597,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 09:24:04.800,2021-01-03 09:25:37.858,consumer,Email,WI


### Dropping duplicate user ids

In [33]:
user_df_cleaned = user_df_cleaned.drop_duplicates(subset='id')
user_df_cleaned.head()

,id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 09:24:04.800,2021-01-03 09:25:37.858,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 09:25:30.554,2021-01-03 09:25:30.597,consumer,Email,WI
6,5ff1e1e8cfcf6c399c274ad9,True,2021-01-03 09:25:28.354,2021-01-03 09:25:28.392,consumer,Email,WI
7,5ff1e1b7cfcf6c399c274a5a,True,2021-01-03 09:24:39.626,2021-01-03 09:24:39.665,consumer,Email,WI
9,5ff1e1f1cfcf6c399c274b0b,True,2021-01-03 09:25:37.564,2021-01-03 09:25:37.599,consumer,Email,WI


### Saving the cleaned User dataframe as a Newline Delimited JSON file to import into BigQuery Database

In [34]:
user_df_cleaned.to_json('user_cleaned.jsonl', orient='records', lines=True)  # Save as records format

In [35]:
print(user_df_cleaned.columns)

Index(['id', 'active', 'createdDate', 'lastLogin', 'role', 'signUpSource',
       'state'],
      dtype='object')


In [36]:
row_count = user_df_cleaned.shape[0]
print(f"Number of rows in the DataFrame: {row_count}")

Number of rows in the DataFrame: 212
